# ⚠️ Prije pokretanja, potrebno je dodati shortcut do projektnog foldera u svoj Google Drive (My Drive) kako bi Colab mogao pristupiti svim potrebnim fajlovima. ⚠️

#Instalacija potrebnih biblioteka i alata


In [38]:
!pip install pretty_midi music21 pygame --quiet
!apt-get install -y fluidsynth > /dev/null

!wget -q https://github.com/musescore/MuseScore/raw/master/share/sound/FluidR3Mono_GM.sf2 -O FluidR3_GM.sf2

print("Sve potrebne biblioteke i alati su instalirani i spremni!")

Sve potrebne biblioteke i alati su instalirani i spremni!


# Uvoz biblioteka i povezivanje s Google Drive-om


In [39]:
import os
import sys
import random
import pretty_midi
import numpy as np
import time
import threading
import queue
import glob
import pickle
import traceback
import wave
import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio, HTML
import matplotlib.pyplot as plt

from music21 import converter, note as m21_note, interval as m21_interval, stream as m21_stream, duration as m21_duration, chord as m21_chord
from collections import Counter

#Povezivanje s Google Drive-om

In [40]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive je uspješno povezan.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive je uspješno povezan.


#Postavke za Collab okruženje

In [41]:
BASE_DIR = '/content/drive/MyDrive/VI projekat/'
DRIVE_MIDI_FOLDER_PATH = os.path.join(BASE_DIR, 'Datasets/Dataset')

SOUND_FONT_FILENAME = 'FluidR3_GM.sf2'
SOUND_FONT_PATH = os.path.join('/content/', SOUND_FONT_FILENAME)

os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(DRIVE_MIDI_FOLDER_PATH, exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'generated_music'), exist_ok=True)

if not os.path.exists(SOUND_FONT_PATH):
    print("UPOZORENJE: SoundFont nije pronađen. Konverzija u WAV neće raditi.")

print(f"Glavni folder projekta: {BASE_DIR}")
print(f"MIDI Dataset putanja: {DRIVE_MIDI_FOLDER_PATH}")

Glavni folder projekta: /content/drive/MyDrive/VI projekat/
MIDI Dataset putanja: /content/drive/MyDrive/VI projekat/Datasets/Dataset


#Konstante

In [42]:
POSSIBLE_DURATIONS = [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 10.0]
DEFAULT_VELOCITY = 100
MIN_PITCH_GA = 40
MAX_PITCH_GA = 84
GA_POPULATION_SIZE = 30
GA_NUM_GENERATIONS = 25
GA_MELODY_LENGTH = 30
GA_MUTATION_RATE = 0.15
GA_CROSSOVER_RATE = 0.7
GA_BPM = 120

GM_INSTRUMENTS = [
    "Acoustic Grand Piano", "Bright Acoustic Piano", "Electric Grand Piano", "Honky-tonk Piano",
    "Electric Piano 1", "Electric Piano 2", "Harpsichord", "Clavinet", "Celesta", "Glockenspiel",
    "Music Box", "Vibraphone", "Marimba", "Xylophone", "Tubular Bells", "Dulcimer", "Drawbar Organ",
    "Percussive Organ", "Rock Organ", "Church Organ", "Reed Organ", "Accordion", "Harmonica",
    "Tango Accordion", "Acoustic Guitar (nylon)", "Acoustic Guitar (steel)", "Electric Guitar (jazz)",
    "Electric Guitar (clean)", "Electric Guitar (muted)", "Overdriven Guitar", "Distortion Guitar",
    "Guitar Harmonics", "Acoustic Bass", "Electric Bass (finger)", "Electric Bass (pick)",
    "Fretless Bass", "Slap Bass 1", "Slap Bass 2", "Synth Bass 1", "Synth Bass 2", "Violin",
    "Viola", "Cello", "Contrabass", "Tremolo Strings", "Pizzicato Strings", "Orchestral Harp",
    "Timpani", "String Ensemble 1", "String Ensemble 2", "Synth Strings 1", "Synth Strings 2",
    "Choir Aahs", "Voice Oohs", "Synth Voice", "Orchestra Hit", "Trumpet", "Trombone", "Tuba",
    "Muted Trumpet", "French Horn", "Brass Section", "Synth Brass 1", "Synth Brass 2", "Soprano Sax",
    "Alto Sax", "Tenor Sax", "Baritone Sax", "Oboe", "English Horn", "Bassoon", "Clarinet", "Piccolo",
    "Flute", "Recorder", "Pan Flute", "Blown Bottle", "Shakuhachi", "Whistle", "Ocarina",
    "Lead 1 (square)", "Lead 2 (sawtooth)", "Lead 3 (calliope)", "Lead 4 (chiff)", "Lead 5 (charang)",
    "Lead 6 (voice)", "Lead 7 (fifths)", "Lead 8 (bass + lead)", "Pad 1 (new age)", "Pad 2 (warm)",
    "Pad 3 (polysynth)", "Pad 4 (choir)", "Pad 5 (bowed)", "Pad 6 (metallic)", "Pad 7 (halo)",
    "Pad 8 (sweep)", "FX 1 (rain)", "FX 2 (soundtrack)", "FX 3 (crystal)", "FX 4 (atmosphere)",
    "FX 5 (brightness)", "FX 6 (goblins)", "FX 7 (echoes)", "FX 8 (sci-fi)", "Sitar", "Banjo",
    "Shamisen", "Koto", "Kalimba", "Bagpipe", "Fiddle", "Shanai", "Tinkle Bell", "Agogo",
    "Steel Drums", "Woodblock", "Taiko Drum", "Melodic Tom", "Synth Drum", "Reverse Cymbal",
    "Guitar Fret Noise", "Breath Noise", "Seashore", "Bird Tweet", "Telephone Ring", "Helicopter",
    "Applause", "Gunshot"
]

#Glavna logika

## Klasa za evaluaciju melodije prema naučenom muzičkom stilu

In [43]:
class StyleEvaluator:

    # Inicijalizacija evaluator-a sa opcionalnim težinama za različite metrike i maksimalnim razmakom u semitonima za upoređivanje intervala
    def __init__(self, weights=None, max_interval_semitones=24):
        self.style_pitch_class_dist = None
        self.style_interval_dist = None
        self.style_pitch_class_bigram_dist = None
        self.style_duration_dist = None
        self.style_ioi_dist = None

        self.weights = weights if weights is not None else {
            "pitch_class_similarity": 0.25,
            "interval_similarity": 0.25,
            "bigram_avg_prob": 0.20,
            "duration_similarity": 0.15,
            "ioi_similarity": 0.15
        }
        self.max_interval_semitones = max_interval_semitones
        self._all_pitch_classes = list(range(12))
        self._all_intervals = list(range(-max_interval_semitones, max_interval_semitones + 1))
        self._all_durations_for_dist = sorted(list(set(POSSIBLE_DURATIONS)))
        self._all_iois_for_dist = sorted(list(set(POSSIBLE_DURATIONS)))

    # Normalizuje frekvencijski `Counter` u distribuciju vjerovatnoća (sumira na 1.0)
    def _normalize_counter(self, counts_counter, all_keys_for_distribution=None):
        total_sum = sum(counts_counter.values())
        if total_sum == 0:
            if all_keys_for_distribution:
                return {k: 0.0 for k in all_keys_for_distribution}
            return {k: 0.0 for k in counts_counter}
        normalized_dist = {k: v / total_sum for k, v in counts_counter.items()}
        if all_keys_for_distribution:
            final_dist = {k: 0.0 for k in all_keys_for_distribution}
            final_dist.update(normalized_dist)
            return final_dist
        return normalized_dist

    # Kvantizuje trajanje note na najbližu vrijednost iz skupa mogućih trajanja
    def _quantize_duration(self, duration, bins=POSSIBLE_DURATIONS):
        if not bins:
            return duration
        return min(bins, key=lambda x: abs(x - duration))

    # Ekstraktuje stilske karakteristike (pitch class, intervali, bigrami, duracije, IOI) iz liste nota
    def _extract_features(self, melody_dict_list):
        pitches_for_style = [note['pitch'] for note in melody_dict_list if note['pitch'] is not None]
        pitch_classes_counts = Counter()
        intervals_counts = Counter()
        pitch_class_bigrams_counts = Counter()
        if pitches_for_style:
            for p in pitches_for_style:
                pitch_classes_counts[p % 12] += 1
            if len(pitches_for_style) > 1:
                for i in range(len(pitches_for_style) - 1):
                    p1, p2 = pitches_for_style[i], pitches_for_style[i + 1]
                    semitones = p2 - p1
                    if abs(semitones) <= self.max_interval_semitones:
                        intervals_counts[semitones] += 1
                    pc1, pc2 = p1 % 12, p2 % 12
                    pitch_class_bigrams_counts[(pc1, pc2)] += 1
        durations_counts = Counter()
        iois_counts = Counter()
        for note_obj in melody_dict_list:
            quantized_dur = self._quantize_duration(float(note_obj['duration']))
            durations_counts[quantized_dur] += 1
            iois_counts[quantized_dur] += 1
        return pitch_classes_counts, intervals_counts, pitch_class_bigrams_counts, durations_counts, iois_counts

    # Uči stil iz kolekcije MIDI fajlova u datom folderu i pravi prosječne distribucije karakteristika
    def learn_style_from_dataset(self, dataset_folder_path):
        print(f"Učim stil iz MIDI fajlova u: {dataset_folder_path}")
        corpus_pc_counts, corpus_interval_counts, corpus_bigram_counts = Counter(), Counter(), Counter()
        corpus_duration_counts, corpus_ioi_counts = Counter(), Counter()
        midi_files = glob.glob(os.path.join(dataset_folder_path, "*.mid")) + glob.glob(os.path.join(dataset_folder_path, "*.midi"))
        if not midi_files:
            print("Nema MIDI fajlova u dataset folderu.")
            return False
        processed_files = 0
        for midi_file in midi_files:
            try:
                score = converter.parse(midi_file)
                current_file_melody_dicts = []
                for element in score.flat.notesAndRests:
                    pitch_val, duration_val = None, 0.0
                    if isinstance(element, m21_note.Note):
                        pitch_val, duration_val = element.pitch.midi, element.duration.quarterLength
                    elif isinstance(element, m21_note.Rest):
                        pitch_val, duration_val = None, element.duration.quarterLength
                    elif isinstance(element, m21_chord.Chord):
                        if element.pitches:
                            pitch_val = max(p.midi for p in element.pitches)
                        else:
                            pitch_val = None
                        duration_val = element.duration.quarterLength
                    else:
                        continue
                    quantized_element_dur = self._quantize_duration(duration_val)
                    current_file_melody_dicts.append({'pitch': pitch_val, 'duration': quantized_element_dur, 'velocity': DEFAULT_VELOCITY})
                if current_file_melody_dicts:
                    pc_c, int_c, bigr_c, dur_c, ioi_c = self._extract_features(current_file_melody_dicts)
                    corpus_pc_counts.update(pc_c)
                    corpus_interval_counts.update(int_c)
                    corpus_bigram_counts.update(bigr_c)
                    corpus_duration_counts.update(dur_c)
                    corpus_ioi_counts.update(ioi_c)
                    processed_files += 1
            except Exception as e:
                print(f"Greška pri obradi fajla {os.path.basename(midi_file)}: {e}")
        if processed_files == 0:
            print("Nijedan MIDI fajl nije uspešno obrađen. Učenje stila neuspešno.")
            return False
        self.style_pitch_class_dist = self._normalize_counter(corpus_pc_counts, self._all_pitch_classes)
        self.style_interval_dist = self._normalize_counter(corpus_interval_counts, self._all_intervals)
        self.style_pitch_class_bigram_dist = self._normalize_counter(corpus_bigram_counts)
        self.style_duration_dist = self._normalize_counter(corpus_duration_counts, self._all_durations_for_dist)
        self.style_ioi_dist = self._normalize_counter(corpus_ioi_counts, self._all_iois_for_dist)
        print(f"Učenje stila završeno. Obrađeno {processed_files} fajlova.")
        return True

    # Izračunava Bhattacharyya koeficijent (sličnost) između dvije distribucije
    def _calculate_bhattacharyya_coefficient(self, dist1_dict, dist2_dict, all_keys):
        bc = 0.0
        for key in all_keys:
            p_k, q_k = dist1_dict.get(key, 0.0), dist2_dict.get(key, 0.0)
            bc += np.sqrt(p_k * q_k)
        return bc

    # Izračunava “fitness” vrijednost melodije na osnovu sličnosti sa naučenim stilom
    def calculate_fitness(self, melody_dict_list):
        if not all([self.style_pitch_class_dist, self.style_interval_dist, self.style_pitch_class_bigram_dist, self.style_duration_dist, self.style_ioi_dist]):
            return 0.0
        if not melody_dict_list:
            return 0.0
        gen_pc_c, gen_int_c, gen_bigram_c, gen_dur_c, gen_ioi_c = self._extract_features(melody_dict_list)
        current_pc_dist = self._normalize_counter(gen_pc_c, self._all_pitch_classes)
        current_interval_dist = self._normalize_counter(gen_int_c, self._all_intervals)
        current_duration_dist = self._normalize_counter(gen_dur_c, self._all_durations_for_dist)
        current_ioi_dist = self._normalize_counter(gen_ioi_c, self._all_iois_for_dist)
        pc_sim = self._calculate_bhattacharyya_coefficient(self.style_pitch_class_dist, current_pc_dist, self._all_pitch_classes)
        int_sim = self._calculate_bhattacharyya_coefficient(self.style_interval_dist, current_interval_dist, self._all_intervals)
        dur_sim = self._calculate_bhattacharyya_coefficient(self.style_duration_dist, current_duration_dist, self._all_durations_for_dist)
        ioi_sim = self._calculate_bhattacharyya_coefficient(self.style_ioi_dist, current_ioi_dist, self._all_iois_for_dist)
        bigram_avg_prob = 0.0
        num_bigrams_in_melody = sum(gen_bigram_c.values())
        if num_bigrams_in_melody > 0 and self.style_pitch_class_bigram_dist:
            total_bigram_log_prob = sum(count * np.log(self.style_pitch_class_bigram_dist.get(bigram, 1e-9)) for bigram, count in gen_bigram_c.items())
            bigram_avg_prob = np.exp(total_bigram_log_prob / num_bigrams_in_melody)
        fitness = (
            self.weights["pitch_class_similarity"] * pc_sim +
            self.weights["interval_similarity"] * int_sim +
            self.weights["duration_similarity"] * dur_sim +
            self.weights["ioi_similarity"] * ioi_sim +
            self.weights["bigram_avg_prob"] * bigram_avg_prob
        )
        return max(0.0, min(100.0, fitness * 100.0))

##Funckije genetičkog algoritma

In [44]:
# Kreira slučajnu notu sa nasumičnim pitch-om, trajanjem i podrazumijevanom jačinom (velocity)
def create_random_note_for_ga():
    return {
        'pitch': random.randint(MIN_PITCH_GA, MAX_PITCH_GA),
        'duration': random.choice(POSSIBLE_DURATIONS),
        'velocity': DEFAULT_VELOCITY
    }

# Generiše nasumičnu melodiju zadane dužine
def create_random_melody_for_ga(length):
    return [create_random_note_for_ga() for _ in range(length)]

# Inicijalizuje početnu populaciju melodija za genetski algoritam
def initialize_population_for_ga(pop_size, melody_length):
    return [create_random_melody_for_ga(melody_length) for _ in range(pop_size)]

# Selekcija roditelja iz populacije koristeći turnirski izbor baziran na fitness vrijednostima
def selection_tournament(population, fitness_scores, tournament_size=3):
    selected_parents = []
    pop_indices = list(range(len(population)))
    num_parents_to_select = len(population)
    actual_tournament_size = min(tournament_size, len(pop_indices))
    if actual_tournament_size == 0:
        return []
    for _ in range(num_parents_to_select):
        current_sample_size = min(actual_tournament_size, len(pop_indices))
        if current_sample_size == 0:
            break
        tournament_contenders_indices = random.sample(pop_indices, current_sample_size)
        best_contender_index_in_pop = max(tournament_contenders_indices, key=lambda idx: fitness_scores[idx])
        selected_parents.append(population[best_contender_index_in_pop])
    return selected_parents

# Jednopunkcijska rekombinacija (crossover) između dvije melodije uz određenu vjerovatnoću
def crossover_one_point(parent1_melody, parent2_melody, crossover_rate, melody_len):
    if random.random() < crossover_rate and melody_len > 1:
        point = random.randint(1, melody_len - 1)
        return (
            parent1_melody[:point] + parent2_melody[point:],
            parent2_melody[:point] + parent1_melody[point:]
        )
    return parent1_melody[:], parent2_melody[:]

# Vrši mutaciju na pitch ili trajanje nota u melodiji sa određenom vjerovatnoćom
def mutate_pitch_duration_for_ga(melody, mutation_rate):
    mutated_melody = []
    for note_dict in melody:
        new_note_dict = note_dict.copy()
        if random.random() < mutation_rate:
            if random.random() < 0.7:
                new_note_dict['pitch'] = random.randint(MIN_PITCH_GA, MAX_PITCH_GA)
            else:
                new_note_dict['duration'] = random.choice(POSSIBLE_DURATIONS)
        mutated_melody.append(new_note_dict)
    return mutated_melody

## Funkcije za konverziju

In [45]:
# Konvertuje listu nota u MIDI fajl sa zadanim instrumentom i tempom
def melody_dict_list_to_midi(melody_dicts, output_filename, instrument_name='Acoustic Grand Piano', bpm=120.0):
    try:
        midi_data = pretty_midi.PrettyMIDI(initial_tempo=float(bpm))
        instrument_program = pretty_midi.instrument_name_to_program(instrument_name)
        instrument = pretty_midi.Instrument(program=instrument_program)
        current_time = 0.0
        for note_data in melody_dicts:
            note_event = pretty_midi.Note(
                velocity=int(note_data['velocity']),
                pitch=int(note_data['pitch']),
                start=current_time,
                end=current_time + float(note_data['duration']) * (60.0 / bpm)
            )
            instrument.notes.append(note_event)
            current_time += float(note_data['duration']) * (60.0 / bpm)
        midi_data.instruments.append(instrument)
        midi_data.write(output_filename)
        return output_filename
    except Exception as e:
        print(f"Greška pri pisanju MIDI fajla {output_filename}: {e}")
        return None

# Konvertuje MIDI fajl u WAV format koristeći FluidSynth i SoundFont (.sf2) fajl.
def convert_midi_to_wav(midi_file_path, wav_file_path, sound_font_sf2=SOUND_FONT_PATH):
    if not os.path.exists(sound_font_sf2):
        print(f"Greška: SoundFont fajl nije pronađen: {sound_font_sf2}")
        return None
    if not os.path.exists(midi_file_path):
        print(f"Greška: MIDI fajl za konverziju nije pronađen: {midi_file_path}")
        return None
    try:
        command = ['fluidsynth', '-ni', sound_font_sf2, midi_file_path, '-F', wav_file_path, '-r', '44100']
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        if os.path.exists(wav_file_path) and os.path.getsize(wav_file_path) > 0:
            return wav_file_path
        else:
            print("Greška: FluidSynth se završio bez greške, ali WAV fajl nije kreiran.")
            return None
    except subprocess.CalledProcessError as e:
        print(f"Greška prilikom konverzije MIDI u WAV. FluidSynth kod greške: {e.returncode}")
        return None
    except Exception as e:
        print(f"Neočekivana greška tokom WAV konverzije: {e}")
        return None

## Funkcije za crtanje grafika

In [46]:
def plot_pitch_class_distribution(style_evaluator, title="Distribucija Klasa Visina Tona (Dataset)"):
    if not style_evaluator or not style_evaluator.style_pitch_class_dist:
        print("Stilski model nije učitan.")
        return

    pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    distribution = style_evaluator.style_pitch_class_dist

    # Sortiranje vrijednosti prema ključevima (0-11)
    probabilities = [distribution.get(i, 0) for i in range(12)]

    plt.figure(figsize=(10, 5))
    plt.bar(pitch_classes, probabilities)
    plt.title(title)
    plt.xlabel("Klasa visine tona")
    plt.ylabel("Vjerovatnoća")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

In [47]:
def plot_interval_distribution(style_evaluator, title="Distribucija Melodijskih Intervala (Dataset)"):
    if not style_evaluator or not style_evaluator.style_interval_dist:
        print("Stilski model nije učitan.")
        return

    distribution = style_evaluator.style_interval_dist
    intervals = sorted(distribution.keys())
    probabilities = [distribution[i] for i in intervals]

    plt.figure(figsize=(12, 6))
    plt.bar([str(i) for i in intervals], probabilities)
    plt.title(title)
    plt.xlabel("Interval (u polutonovima)")
    plt.ylabel("Vjerovatnoća")
    plt.xticks(rotation=90, fontsize=8)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

In [48]:
def plot_duration_distribution(style_evaluator, title="Distribucija Trajanja Nota (Dataset)"):
    if not style_evaluator or not style_evaluator.style_duration_dist:
        print("Stilski model nije učitan.")
        return

    distribution = style_evaluator.style_duration_dist
    durations = sorted(distribution.keys())
    probabilities = [distribution[d] for d in durations]

    plt.figure(figsize=(10, 5))
    plt.bar([str(d) for d in durations], probabilities)
    plt.title(title)
    plt.xlabel("Trajanje (u četvrtinkama)")
    plt.ylabel("Vjerovatnoća")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

In [49]:
def plot_pitch_class_bigram_heatmap_matplotlib(style_evaluator, title="Vjerovatnoće Tranzicija Nota (Heatmap)"):
    if not style_evaluator or not style_evaluator.style_pitch_class_bigram_dist:
        print("Stilski model nije učitan.")
        return

    bigram_dist = style_evaluator.style_pitch_class_bigram_dist
    pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

    matrix_data = np.zeros((12, 12))

    for (pc1, pc2), prob in bigram_dist.items():
        if pc1 is not None and pc2 is not None:
             matrix_data[pc1, pc2] = prob

    fig, ax = plt.subplots(figsize=(10, 8))

    im = ax.imshow(matrix_data, cmap='viridis', interpolation='nearest')

    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("Vjerovatnoća", rotation=-90, va="bottom")

    ax.set_xticks(np.arange(len(pitch_classes)))
    ax.set_yticks(np.arange(len(pitch_classes)))
    ax.set_xticklabels(pitch_classes)
    ax.set_yticklabels(pitch_classes)

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    ax.set_title(title)
    ax.set_xlabel("Sljedeća nota")
    ax.set_ylabel("Trenutna nota")

    ax.set_xticks(np.arange(matrix_data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(matrix_data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=1)
    ax.tick_params(which="minor", bottom=False, left=False)

    fig.tight_layout()
    plt.show()

In [50]:
def plot_all_statistics(style_evaluator):
  plot_pitch_class_distribution(style_evaluator)
  print()
  plot_interval_distribution(style_evaluator)
  print()
  plot_duration_distribution(style_evaluator)
  print()
  plot_pitch_class_bigram_heatmap_matplotlib(style_evaluator)

#Interfejs

## Definicija widget-a

In [51]:
style = {'description_width': 'initial'}
control_layout = widgets.Layout(width='500px')
button_layout = widgets.Layout(width='200px', height='40px')

# GA Parametri
population_slider = widgets.IntSlider(value=GA_POPULATION_SIZE, min=10, max=200, step=10, description='Populacija:', style=style, layout=control_layout)
generations_slider = widgets.IntSlider(value=GA_NUM_GENERATIONS, min=5, max=200, step=5, description='Generacije:', style=style, layout=control_layout)
melody_length_slider = widgets.IntSlider(value=GA_MELODY_LENGTH, min=5, max=100, step=1, description='Dužina melodije:', style=style, layout=control_layout)
mutation_rate_slider = widgets.FloatSlider(value=GA_MUTATION_RATE, min=0.0, max=1.0, step=0.01, description='Stopa mutacije:', readout_format='.2f', style=style, layout=control_layout)
crossover_rate_slider = widgets.FloatSlider(value=GA_CROSSOVER_RATE, min=0.0, max=1.0, step=0.01, description='Stopa ukrštanja:', readout_format='.2f', style=style, layout=control_layout)
bpm_slider = widgets.IntSlider(value=GA_BPM, min=30, max=240, step=5, description='BPM:', style=style, layout=control_layout)
instrument_dropdown = widgets.Dropdown(options=GM_INSTRUMENTS, value='Acoustic Grand Piano', description='Instrument:', style=style, layout=control_layout)

# Dugmadi i output prozori
generate_button = widgets.Button(description="🎶 Generiraj Melodiju", button_style='success', icon='music', layout=button_layout)
load_style_button = widgets.Button(description="🔄 Učitaj iz Dataseta", button_style='info', icon='book', layout=button_layout)
import_style_button = widgets.Button(description="📥 Učitaj Model (.pkl)", button_style='primary', icon='upload', layout=button_layout)
export_style_button = widgets.Button(description="💾 Eksportuj Model (.pkl)", button_style='warning', icon='save', disabled=True, layout=button_layout)
log_output = widgets.Output(layout={'border': '1px solid black', 'height': '1000px', 'overflow_y': 'scroll'})
audio_output = widgets.Output()

style_evaluator = None

## Funkcije za upravljanje interfejsom

In [52]:
# Postavlja stanja dugmadi
def set_buttons_state(is_busy):
    generate_button.disabled = is_busy or not style_evaluator
    load_style_button.disabled = is_busy
    import_style_button.disabled = is_busy
    export_style_button.disabled = is_busy or not style_evaluator

# Učitava stil iz MIDI datoteka u Dataset folderu
def on_load_style_button_clicked(b):
    global style_evaluator
    with log_output:
        log_output.clear_output(wait=True)
        print("--- Učitavanje stila iz dataseta na Google Driveu ---")
        set_buttons_state(is_busy=True)
        evaluator = StyleEvaluator()
        success = evaluator.learn_style_from_dataset(DRIVE_MIDI_FOLDER_PATH)
        if success:
            style_evaluator = evaluator
            plot_all_statistics(style_evaluator)
            print("Stilski model je uspješno naučen/ažuriran.")
        else:
            style_evaluator = None
            print("Greška: Nije bilo moguće naučiti stilski model. Provjerite jesu li MIDI datoteke u Dataset folderu.")
        set_buttons_state(is_busy=False)

# Učitava prethodno spremljeni .pkl stilski model
def on_import_style_button_clicked(b):
    global style_evaluator
    from google.colab import files
    with log_output:
        log_output.clear_output(wait=True)
        print("--- Učitavanje .pkl stilskog modela ---")
        print("Molimo odaberite .pkl datoteku s vašeg računara...")
        set_buttons_state(is_busy=True)
        uploaded = files.upload()
        if not uploaded:
            print("Učitavanje otkazano.")
            set_buttons_state(is_busy=False)
            return
        filename = next(iter(uploaded))
        if not filename.endswith('.pkl'):
            print(f"Greška: Odabrana datoteka '{filename}' nije .pkl datoteka.")
            set_buttons_state(is_busy=False)
            return
        print(f"Datoteka '{filename}' učitana. Parsiram model...")
        try:
            data = pickle.loads(uploaded[filename])
            evaluator = StyleEvaluator()
            evaluator.style_pitch_class_dist = data.get('pc_dist')
            evaluator.style_interval_dist = data.get('int_dist')
            evaluator.style_pitch_class_bigram_dist = data.get('bigr_dist')
            evaluator.style_duration_dist = data.get('dur_dist')
            evaluator.style_ioi_dist = data.get('ioi_dist')
            evaluator.weights = data.get('weights', evaluator.weights)
            if all([evaluator.style_pitch_class_dist, evaluator.style_interval_dist, evaluator.style_pitch_class_bigram_dist]):
                style_evaluator = evaluator
                plot_all_statistics(style_evaluator)
                print(f"Stilski model uspješno učitan iz: {filename}")
            else:
                style_evaluator = None
                print(f"Greška: Učitani model iz {filename} je nekompletan.")
        except Exception as e:
            style_evaluator = None
            print(f"Kritična greška pri učitavanju modela: {e}")
        set_buttons_state(is_busy=False)

# Sprema trenutni stilski model u .pkl datoteku na Google Drive
def on_export_style_button_clicked(b):
    global style_evaluator
    with log_output:
        if not style_evaluator:
            print("Nema naučenog stilskog modela za eksport.")
            return
        file_path = os.path.join(BASE_DIR, 'Models/learned_style_model.pkl')
        try:
            data = {'pc_dist': style_evaluator.style_pitch_class_dist, 'int_dist': style_evaluator.style_interval_dist, 'bigr_dist': style_evaluator.style_pitch_class_bigram_dist, 'dur_dist': style_evaluator.style_duration_dist, 'ioi_dist': style_evaluator.style_ioi_dist, 'weights': style_evaluator.weights}
            with open(file_path, 'wb') as f:
                pickle.dump(data, f)
            print(f"Stilski model uspješno eksportiran u: {file_path}")
        except Exception as e:
            print(f"Greška pri eksportiranju: {e}")

# Pokreće genetski algoritam za stvaranje melodije
def on_generate_button_clicked(b):
    global style_evaluator
    with log_output:
        log_output.clear_output(wait=True)
        audio_output.clear_output()
        if not style_evaluator:
            print("Greška: Stilski model nije učitan.")
            return
        set_buttons_state(is_busy=True)
        print("--- Pokretanje Genetskog Algoritma ---")
        pop_size, gens, mel_len, mut, cross, bpm, instr = population_slider.value, generations_slider.value, melody_length_slider.value, mutation_rate_slider.value, crossover_rate_slider.value, bpm_slider.value, instrument_dropdown.value
        print(f"Parametri: Pop={pop_size}, Gen={gens}, Len={mel_len}, BPM={bpm}, Instr={instr}")

        population = initialize_population_for_ga(pop_size, mel_len)
        best_fit, best_genome = -1.0, None
        for gen in range(gens):
            scores = [style_evaluator.calculate_fitness(mel) for mel in population]
            if np.max(scores) > best_fit:
                best_fit = np.max(scores)
                best_genome = population[np.argmax(scores)][:]
            print(f"Gen {gen + 1}/{gens} - Najbolji fitness: {best_fit:.2f}")
            parents = selection_tournament(population, scores)
            next_pop = [best_genome[:]] if best_genome else []
            while len(next_pop) < pop_size:
                p1, p2 = random.sample(parents, 2)
                c1, c2 = crossover_one_point(p1, p2, cross, mel_len)
                if len(next_pop) < pop_size: next_pop.append(mutate_pitch_duration_for_ga(c1, mut))
                if len(next_pop) < pop_size: next_pop.append(mutate_pitch_duration_for_ga(c2, mut))
            population = next_pop
        print(f"\nGA završen. Najbolji fitness: {best_fit:.2f}")

        if best_genome:
            print("Generiram audio...")
            out_dir = os.path.join(BASE_DIR, "generated_music")
            f_base = f"gen_mel_{time.strftime('%Y%m%d-%H%M%S')}"
            mid_file = melody_dict_list_to_midi(best_genome, os.path.join(out_dir, f"{f_base}.mid"), instr, bpm)
            if mid_file:
                print(f"MIDI sačuvan: {os.path.basename(mid_file)}")
                wav_file = convert_midi_to_wav(mid_file, os.path.join(out_dir, f"{f_base}.wav"))
                if wav_file:
                    print(f"WAV sačuvan: {os.path.basename(wav_file)}")
                    with audio_output: display(Audio(wav_file))
        set_buttons_state(is_busy=False)

## Prikaz interfejsa

In [53]:
# Povezivanje funkcija s događajima
load_style_button.on_click(on_load_style_button_clicked)
import_style_button.on_click(on_import_style_button_clicked)
generate_button.on_click(on_generate_button_clicked)
export_style_button.on_click(on_export_style_button_clicked)

ga_params_box = widgets.VBox([
    population_slider, generations_slider, melody_length_slider,
    mutation_rate_slider, crossover_rate_slider, bpm_slider, instrument_dropdown
])
buttons_box = widgets.HBox([load_style_button, import_style_button, generate_button, export_style_button])
accordion = widgets.Accordion(children=[ga_params_box])
accordion.set_title(0, 'GA Parametri i Postavke')

# Glavni naslov i upute
display(HTML("<h2>Stilski Muzički Generator (Colab verzija)</h2>"))
display(HTML("""
<p>
<b>1. Pripremite Stil:</b> Imate dvije opcije:<br>
    <b>A) Iz Dataseta:</b> Postavite MIDI datoteke u <code>/Colab_Music_Generator/Dataset/</code> na vašem Google Driveu i kliknite <b>'Učitaj iz Dataseta'</b>.<br>
    <b>B) Iz Modela:</b> Kliknite <b>'Učitaj Model (.pkl)'</b> i odaberite prethodno spremljenu .pkl datoteku s vašeg računara.<br>
<b>2. Generirajte muziku:</b> Kada je stil učitan, prilagodite parametre i kliknite <b>'Generiraj Melodiju'</b>.<br>
<b>3. Spremanje:</b> Generirana muzika i eksportirani modeli spremaju se u folder <code>/generated_music/</code> na vašem Driveu.
</p>
"""))

# Prikaz svih elemenata
display(buttons_box)
display(accordion)
display(HTML("<h3>Audio Player</h3>"))
display(audio_output)
display(HTML("<h3>Logovi</h3>"))
display(log_output)

# Postavljanje inicijalnog stanja aplikacije
with log_output:
    generate_button.disabled = True
    print("Dobrodošli! Molimo prvo učitajte stil (iz dataseta ili .pkl datoteke).")

Accordion(children=(VBox(children=(IntSlider(value=30, description='Populacija:', layout=Layout(width='500px')…

Output()

Output(layout=Layout(border='1px solid black', height='1000px', overflow_y='scroll'))